In [1]:
from tennis_tracker.download_data.extract_keypoints import read_json_file
import os

FILE_PATH = "/Users/derek/Desktop/tennis_tracker/tennis_tracker/psudeo_label/clean_labels.json"

data = read_json_file(FILE_PATH)

key1 = list(data.keys())[0]
actual_key = key1.split("/tennis_tracker/")[1]
actual_path = os.path.join("/Users/derek/Desktop/tennis_tracker", actual_key).replace(" &", "&")

In [33]:
import cv2
import numpy as np

FILE_PATH = "/Users/derek/Desktop/tennis_tracker/tennis_tracker/psudeo_label/clean_labels.json"
FPS = 30
FRAME_WIDTH, FRAME_HEIGHT = 640, 480
COURT_HEIGHT = 799
COURT_WIDTH = 560

def get_next_frame(data_entry: dict, court_image: np.array):
    """We want to get tracked points and draw them on the court image"""
    tracked_points = data_entry['transformed_coords']
    original_image = cv2.imread(data_entry['actual_path'])
    original_image = cv2.resize(original_image, (640, 480))
    for point in tracked_points:
        cv2.circle(court_image, point, 10, (0, 0, 255), -1)
    return original_image, tracked_points


data = read_json_file(FILE_PATH)

tiafoe_video_frames = []
keys = list(data.keys())
for key in keys:
    if "Tiafoe" in key:
        tiafoe_video_frames.append((key, int(key.split("/")[-1].split(".")[0])))

tiafoe_video_frames.sort(key=lambda x: x[1])
tiafoe_video_frames = [x[0] for x in tiafoe_video_frames]



output_video = cv2.VideoWriter('tennis_tracking_output.avi', cv2.VideoWriter_fourcc(*'XVID'), FPS, (FRAME_WIDTH + COURT_WIDTH, FRAME_HEIGHT + COURT_HEIGHT))
for i in range(len(data)):
    court_image = cv2.imread("/Users/derek/Desktop/tennis_tracker/tennis_tracker/player_location/padded_court.jpg")
    original_frame, tracked_points = get_next_frame(data[i])
    combined_frame = np.hstack((court_image, original_frame))
    cv2.imshow("Tennis Tracking", combined_frame)
    output_video.write(combined_frame)
    if cv2.waitKey(int(1000 / FPS)) & 0xFF == ord('q'):
        break


[360, 640]